In [3]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax

from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

label = train['class']

char_embedding_index = {}
f = open('word_embedding_300dim_new2.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0].lower()
    embeddings = np.asarray(values[1:], dtype = 'float32')
    char_embedding_index[word] = embeddings
f.close()

len(char_embedding_index)

api_embedding_file_path = 'word_embedding_300dim_new2.txt'
embedding_dim = 300
max_nb_api = len(char_embedding_index) + 1
max_sequence_length_api = 1000

all_word = list(train['word_seg'].values) + list(test['word_seg'].values)

all_word_list = []
for i in list(all_word):
    all_word_list.append(i.split(' '))

len(all_word_list)

train['word_seg'].apply(lambda x:len(x.split(' '))).describe()

data = pd.DataFrame()

gc.collect()

tokenizer_api = Tokenizer()

tokenizer_api.fit_on_texts(all_word_list)

api_seq = tokenizer_api.texts_to_sequences(all_word_list)

data['word_seq'] = pad_sequences(api_seq, maxlen = max_sequence_length_api, padding='post').tolist()

api_index = tokenizer_api.word_index
print('Found %s unique tokens' % len(api_index))
nb_words = min(max_nb_api, len(char_embedding_index)+1)
api_embedding_matrix = np.zeros((nb_words, embedding_dim))

for word, i in tqdm(api_index.items()):
    embedding_vector = char_embedding_index.get(word)
    if embedding_vector is not None:
        api_embedding_matrix[i] = embedding_vector
    else:
        print('no char%s'%word)

data['id'] = train['id']

train_FE = data.iloc[:len(label),:]
test_FE = data.iloc[len(label):,:]

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

def get_input(df):
    
    _input = [np.array(df.word_seq.values.tolist())]
    
    
    return _input 

/home/libo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
1271461it [02:09, 9784.64it/s]
  1%|▏         | 17850/1271460 [00:00<00:07, 178445.71it/s]

Found 1271460 unique tokens


100%|██████████| 1271460/1271460 [00:29<00:00, 43778.21it/s]


In [2]:
def CNN1():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    char_cnn = api_embedding_layer(char_input)
    #char_cnn = SpatialDropout1D(0.2)(char_cnn)
    
    kernel_sizes = [1, 2, 3, 4, 5, 6, 7]
    pooled_word_q1 = []

    for kernel in kernel_sizes:
            
        conv_word_q1 = Conv1D(filters=300,
                          kernel_size=kernel,
                          padding='same',
                          strides=1, 
                          kernel_initializer='he_uniform',
                          activation='relu')(char_cnn)

        pool_word_q1 = MaxPooling1D(pool_size = max_sequence_length_api)(conv_word_q1)

        pooled_word_q1.append(pool_word_q1)

    merged_cnn_word_q1 = Concatenate(axis=-1)(pooled_word_q1)

    flatten_cnn_word_q1 = Flatten()(merged_cnn_word_q1)
    
    merged = Dropout(0.2)(flatten_cnn_word_q1)
    merged = BatchNormalization()(merged)
    merged = Dense(300, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [4]:
label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [5]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

In [6]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
#     bst_model_path = '5folds_' + 'dae_nn_v1_' + str(i) + '.hdf5'
#     early_stopping =EarlyStopping(monitor='val_loss', patience=10)
#     model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
#     callbacks = [
#             early_stopping,
#             model_checkpoint
#         ]
    model = CNN1()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=3, batch_size=BATCH_SIZE, shuffle=True,)#callbacks=callbacks
    
#     model = NN()
#     model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/3
92040/92040 [==============================] - 323s 4ms/step - loss: 1.0301 - acc: 0.7015 - val_loss: 0.8068 - val_acc: 0.7593
Epoch 2/3
92040/92040 [==============================] - 320s 3ms/step - loss: 0.7399 - acc: 0.7782 - val_loss: 0.7850 - val_acc: 0.7674
Epoch 3/3
102277/102277 [==============================] - 91s 891us/step
*******1*******
Train on 92043 samples, validate on 10234 samples
Epoch 1/3
92043/92043 [==============================] - 331s 4ms/step - loss: 1.0248 - acc: 0.7020 - val_loss: 0.8385 - val_acc: 0.7563
Epoch 2/3
92043/92043 [==============================] - 317s 3ms/step - loss: 0.7378 - acc: 0.7790 - val_loss: 0.7990 - val_acc: 0.7650
Epoch 3/3
102277/102277 [==============================] - 90s 876us/step
*******2*******
Train on 92044 samples, validate on 10233 samples
Epoch 1/3
92044/92044 [==============================] - 337s 4ms/step - loss: 1.0359 - acc: 0.7005 - val_

In [11]:
meta_train.columns = ['meta_cnn_word_v1_' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [20]:
meta_train.to_csv('/home/libo/daguan/stack/train_meta_cnn_word_v1.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack/test_meta_cnn_word_v1.csv', index = None ,encoding = 'utf-8')

In [14]:
meta_test.shape

(102277, 190)

In [12]:
meta_train

,meta_cnn_word_v1_prob0,meta_cnn_word_v1_prob1,meta_cnn_word_v1_prob2,meta_cnn_word_v1_prob3,meta_cnn_word_v1_prob4,meta_cnn_word_v1_prob5,meta_cnn_word_v1_prob6,meta_cnn_word_v1_prob7,meta_cnn_word_v1_prob8,meta_cnn_word_v1_prob9,meta_cnn_word_v1_prob10,meta_cnn_word_v1_prob11,meta_cnn_word_v1_prob12,meta_cnn_word_v1_prob13,meta_cnn_word_v1_prob14,meta_cnn_word_v1_prob15,meta_cnn_word_v1_prob16,meta_cnn_word_v1_prob17,meta_cnn_word_v1_prob18
0,0.784283,0.00611387,4.19331e-05,0.000208416,0.000237291,0.000838778,0.00180459,0.00616606,0.000135466,0.00505945,0.000329468,0.00227446,0.00374275,0.138949,0.000569641,0.0450255,0.00129429,0.000753305,0.0021718
1,0.00674082,0.00628316,0.285769,0.0775071,0.000404844,0.0454974,0.00457339,0.00085662,0.00571229,0.000944066,0.00384913,0.0408608,0.0648919,0.000286175,0.0935788,0.000742015,0.000284937,0.0722445,0.288974
2,0.0104989,0.000460044,0.00300776,0.00592548,0.00251502,0.00221117,0.117751,0.00500005,0.00136235,0.00666911,0.0694839,0.0273004,0.0172299,0.689564,0.0121447,8.03311e-05,0.0172513,0.00182287,0.00972164
3,0.00803261,2.86453e-05,1.54314e-07,0.0011635,0.000663891,1.51251e-06,5.70341e-05,0.000175953,0.000130978,1.29468e-05,1.17394e-05,1.0733e-05,0.769339,0.207991,2.71567e-05,0.0116583,0.000210378,1.6062e-05,0.000468728
4,0.0352527,0.000972315,0.00388443,0.146941,0.000646323,0.00194586,0.477683,0.000320832,0.00367274,0.000949624,0.00475928,0.289051,0.0114878,0.0169894,0.00120854,0.000611367,0.00197988,0.000122023,0.00152248
5,0.0057125,2.38464e-05,0.000208857,2.8694e-05,1.18612e-05,1.07754e-05,3.53893e-06,9.36808e-06,2.00293e-06,5.57724e-06,0.000644598,0.000136217,0.972048,7.10143e-06,0.00172283,0.019339,5.00103e-06,3.78546e-07,7.97678e-05
6,0.41581,0.015038,0.0472876,0.0158189,0.0105305,0.00583263,0.00457243,0.00405853,0.00678282,0.26861,0.115498,0.0247689,0.00233177,0.0368609,0.00801485,0.000285015,0.00276868,0.0127652,0.00236603
7,2.14592e-05,0.000118626,0.000784784,2.21511e-05,0.000177434,0.0544505,5.08013e-05,0.000129287,1.42904e-05,0.943951,4.88027e-05,6.05446e-05,3.06279e-05,3.50517e-05,1.65662e-05,3.10954e-05,2.51258e-05,5.65935e-06,2.62239e-05
8,0.00296531,0.00141112,3.43279e-05,6.02607e-05,0.00385458,8.49661e-05,0.00159665,0.00219681,3.46394e-05,0.985292,0.000579687,0.00100335,0.000103064,0.000478383,2.49074e-05,7.97322e-05,0.000139048,6.43104e-06,5.47374e-05
9,0.0721598,0.0018074,0.030037,0.00126617,0.000325267,0.000235074,0.00683498,0.000357459,0.0691068,0.000303539,0.0181514,0.0683039,0.0343654,0.0583894,0.0010457,0.000159311,0.000283276,0.00551737,0.631351
